In [ ]:
# Import
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

df = pd.read_csv('')

In [ ]:
# Explore
print(df.shape)
df.isna().sum()

In [ ]:
# Data Cleaning if needed

# df.reset_index(inplace=True) if id is set as index value before line is ran

In [ ]:
# Modeling
# May need to add our own dictionary
# Create a TfidfVectorizer and Remove stopwords
tfidf = TfidfVectorizer(stop_words='english')
        
# Fit and transform the data to a tfidf matrix
tfidf_matrix = tfidf.fit_transform(df_articles['text'])
        
# Print the shape of the tfidf_matrix
print(tfidf_matrix.shape)

In [ ]:
# Deployment
# Compute the cosine similarity between each movie description
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
      
# For easier viewing, put it in a dataframe
pd.DataFrame(cosine_sim)

df_results = pd.DataFrame(cosine_sim, columns=df['Id'], index=df_articles['Id']) #.sort_values(by=[0], ascending=False)

df_results.to_csv('content_recs.csv')